# Rethinking Food Recommendations
CS 4501: Markets, Mechanisms, and Machines Final Project

Grace Wu (gw5cc), Barry Chin (bxc2gn), Youbeen Shim (ys3ep)

## Introduction

Our project stemmed from the question: “Why are restaurant recommendations not based on whether or not people similar to us have enjoyed it?” The only information that is given to customers looking for new restaurants is an average rating (stars), distance from the customer, and the price range that is offered. However, given today’s technology and data, it is possible to build a more comprehensive algorithm that gives a more customized recommendation based on other users who have a similar taste to the customer using data such as age, gender, budget, or the customers’ preferences.

We explored several different approaches to designing such an algorithm, including Regression Analysis (supervised machine learning) and Clustering (unsupervised machine learning). In Regression Analysis, our focus was finding variables that were significant in relation to how well a user rates a restaurant (ranging from zero to two stars). In Clustering however, we focused more on finding the optimum number of clusters, or groups of individuals who had similar tastes. These two approaches in tandem formed the backbone for structuring our restaurant recommendation algorithm. 


## Data and Processing

### Background

The data, or the collection of datasets, is called “Restaurant & Consumer Data” by Refael Ponce Medellin and Gabriel González Serna at National Center for Research and Technological Development CENIDET, México.

The dataset consisted of 9 .csv files, as follows: 
Restaurant - `chefmozaccepts.csv, chefmozcuisine.csv, chefmozhours4.csv, chefmozparking.csv, geoplaces2.csv`
Users - `usercuisine.csv, userpayment.csv, userprofile.csv`
User-Item-Rating - `rating_final.csv`.


In [ ]:
# Libraries
library(tidyverse)
library(cluster)

# Data
u_cuisine <- read_csv("usercuisine.csv") %>% na.omit()
r_cuisine <- read_csv("chefmozcuisine.csv") %>% na.omit()
rating <- read_csv("rating_final.csv") %>% na.omit()
u_features <- read_csv("userprofile.csv") %>% na.omit()





Considering the constraints of both time and scope, we have elected to use four different .csv files. `Chefmozcuisine.csv` contains the data regarding the types of food that the restaurant served. `Usercuisine.csv` contains the data regarding the types of food that the user indicated that he/she enjoys. `Userprofile.csv` contains the data of the survey responses of the user as they were creating the online user profile. Lastly, `rating_final.csv` contains the data of a users’ rating of a particular restaurant. 



In [ ]:
# Data Cleaning & Organizing
# orgRating - User, Place, Rating, Matching
orgRating <- rating %>%
  inner_join(r_cuisine, by="placeID") %>%
  inner_join(u_cuisine, by="userID") %>%
  mutate(matching=ifelse(Rcuisine.x==Rcuisine.y, 1, 0)) %>%
  group_by(userID, placeID) %>%
  arrange(desc(matching)) %>%
  filter(row_number()==1L) %>%
  select(userID, placeID, rating, matching)
orgRating

# withFeature - orgRating plus user description
withFeature <- orgRating %>%
  inner_join(u_features, by='userID') %>%
  mutate(smoker=ifelse(smoker=='true', 1, 0)) %>%
  select(-latitude, -longitude)
withFeature



### Merging and Transformation

Transformation and merging of the data happened prior to the analysis and characterization using the R package Tidyverse, especially the Dplyr package. RData orgRating was created using `usercuisine.csv`, `chefmozcuisine.csv`, and `rating_final.csv`. Later, withFeature RData was created using orgRating.RData and `userprofile.csv`. Both were used for regression analysis stage of our project. Few functions in the code are worth noting: inner_join was used to merge the different data, mutate to create new column values based on conditions and for feature engineering, and groupby was crucial for giving proper structure to the data.

The RData horizontal was created based on the previously imported and transformed data, especially `final_rating.csv` and `chefmozcuisine.csv`. Specifically, we grouped the merge of the rating and restaurant data by UserID (identifier for users) and the Rcuisine (the column specifying the type of food that the user was served). Then, count (count of type of food that the user was served) and avg (average rating that the user gave for the restaurant serving that type of food). Then, we converted the vertical values of frequency and average rating using groupby to horizontal column space where each column was the type of food served. 


In [ ]:
# whereYoubeen - Places the user has been to & what they served
whereYoubeen <- rating %>%
  inner_join(r_cuisine, by='placeID') %>%
  select(userID, placeID, Rcuisine, rating)
whereYoubeen

# freqVisit - User, type of food they go for, and how frequently (+ avg rating for that type of food)
freqVisit <- whereYoubeen %>%
  group_by(userID, Rcuisine) %>%
  mutate(count=n(), avg=mean(rating)) %>%
  select(userID, Rcuisine, count, avg) %>%
  distinct(genre_rating_group, .keep_all = TRUE)

# horizontal - freqVisit with types of food as columns
horizontal <- freqVisit %>% 
  select(userID, Rcuisine, count) %>%
  spread(Rcuisine, count, fill=0) %>% na.omit()

# Export Data
write.csv(orgRating, file = "orgRating.csv")
write.csv(withFeature, file = "withFeature.csv")

## Implementation 

### Regression Analysis

As discussed above in the Data & Processing section, multiple data sets were merged specifically to build a Ordinary Least Squares regression. The goal of the OLS regression was not only to build the most accurate model, but to build a model that is easily interpretable and isolate features in a way that tells a relationship between the features and the overall rating. Thus, features such as religion and favorite colors were thrown out prior to building the model. 

Before building the OLS regression model, other transformations and processing of the data were required. Some values were left null, suggesting the user did not provide complete data. Only users with complete profiles for the features we desired were selected for our regression. This suggests that further studies with more robust data should follow our study. The data we were working on also heavily consisted of quantitative data, requiring minor feature engineering and regression techniques. Features with two responses, such as “smoker” or “non-smoker” were transformed into numeric binary values. Features with multiple responses, such as “casual drinking”, “social drinking”, and “non drinker” were simplified to binary value of 1 for “drinker” and 0 for “non drinker”. Dummy variables were created for quantitative features with multiple values that could not be simplified further. Ultimately, following variables were considered: ‘budget’, ‘drink’, ‘activity’, ‘smoker’, ‘marital_status’, ‘dress_preference’, ‘rating’, and ‘matching’. OLS regression was run with the ‘rating’ value as the dependent variable. 

Of the multiple variables that were tested in our regression, the variables that proved significant based on their p-values were ‘matching’, ‘budget’, ‘activity’, ‘drinker’, and ‘dress preference’. It was surprising how ‘matching’ had a negative coefficient and how ‘budget_medium’ had a higher coefficient than ‘budget_high’. The negative coefficient can be attributed to the fact that people whose cuisine type preference matched that of the restaurant had greater expectations, thus less willing to give higher ratings. The same goes for why ‘budget_medium’ had a greater effect than ‘budget_high’. It was also reasonable to see that people who preferred to dress formally gave higher ratings because they are more likely to dine at high end restaurants. The regression also had a R-squared value of .706. This isn’t incredibly high, but the results are still good because a high R-squared value isn’t necessary for recommendations. The resulting R-square value also provided a variance of .8402.


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Apr 29 18:00:26 2019

@author: barrychin
"""


import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats
# The important variables: budget, activity, marital_status, smoker, drink level, dress_preference  

#Reading the files 
df_features = pd.read_csv('RestaurantData/withFeature.csv')
df_features = df_features[['userID', 'rating', 'matching', "budget", 'activity', 'marital_status', 'smoker', 'drink', "dress_preference"]]

df_uCuisine = pd.read_csv('RestaurantData/usercuisine.csv')
df_uCuisine = df_uCuisine[['userID', 'Rcuisine']]

df_geoplaces = pd.read_csv('RestaurantData/geoplaces2.csv', encoding='ISO-8859-1')
df_geoplaces = df_geoplaces[['placeID', 'name']]

#Merging data
df_features = pd.merge(df_features, df_uCuisine, on=['userID'])

#Creating Dummy Variables
df_features = pd.get_dummies(df_features, columns=['budget'], prefix = ['budget'])
df_features = pd.get_dummies(df_features, columns=['activity'], prefix = ['activity'])
df_features = pd.get_dummies(df_features, columns=['marital_status'], prefix = ['marital_status'])
df_features = pd.get_dummies(df_features, columns=['dress_preference'], prefix = ['dress_preference'])

#OLS Regression
x_feature = df_features[['matching', "budget_low", 'budget_medium', 'budget_high', 'activity_student', 'activity_professional', 'activity_unemployed', 'activity_working-class', 'marital_status_single', 'marital_status_married', 'smoker', 'drink', "dress_preference_informal", "dress_preference_no preference", "dress_preference_formal"]]
y_feature = df_features[['rating']]

model = sm.OLS(y_feature, x_feature).fit()  # fit the model
print(model.summary()) 

#2 Sample t-test on ratings based on matching
df_ratings = pd.read_csv('RestaurantData/withFeature.csv')
df_ratings = df_ratings['rating']

print(stats.ttest_ind(df_ratings[0:219], df_ratings[220:]))



This was the result of our regression analysis: 
<br> 
<br>
<img src="OLS.png"> 


A 2 sample t-test was also performed to compare the ratings among reviews that had matched preferences to those that didn’t. It was important to analyze this to determine how strong of a factor matching preferences is. The end result was a p-value of .0181, which meant that the mean difference between the two groups is significantly different. 

<br>
<br>
<img src="t_test_result.png"> 

### Clustering

Our second approach was to use k-means clustering to analyze our restaurant recommendation problem. We made several assumptions in regards to this analysis: 

- There are groups of individuals with similar preferences for food.
- The user is more likely to conform to their preferences and pick a restaurant that is similar to their current tastes. 

In this case, we did not look into the overall “rating” since we did not need to predict whether the user would rate the restaurant highly; we simply needed to predict which restaurant the user may be inclined to go to. 

#### Analysis and Insight

In k-means clustering, the number of clusters are arbitrary; the ideal number of clusters can be found by looking at the between sum of squares and within sum of squares value. Between sum of squares refers to the distance between the different type of clusters while within sum of squares refers to the distance of the points within clusters. In this case, we want the between sum of squares - the distance between the different clusters - to be high and the within sum of squares - the distance of the individual points within the clusters - to be low. 

#### Code

Below is the output of our code. This part of our project was done in R. The code outputs the between sum of squares and within sum of squares graphs. 

In [ ]:


# Type of Food offered by Restaurants
unique(r_cuisine$Rcuisine) # Approx. 60 Types of Food offered

# Testing Clustering - pick the best k: minimize withinss, maximize betweenss
output <- matrix(ncol=3, nrow=30)
for (i in 1:20) {
  output[i,1] <- i
  km <- kmeans(horizontal[c(2:4,6:19,21:24)], i, nstart=20)
  output[i,2] <- mean(km$withinss)
  output[i,3] <- mean(km$betweenss)
}
colnames(output) <- c('cluster', 'within', 'between')
output <- data.frame(output)

ggplot(output, aes(x=cluster, y=within)) + geom_point() 
ggplot(output, aes(x=cluster, y=between)) + geom_point()

km <- kmeans(horizontal[c(2:4,6:19,21:24)], 6, nstart=20)
k_cluster <- km$centers

# Testing Clustering 2
horizontal2 <- freqVisit %>%
  mutate(present=ifelse(count>0, 1, 0)) %>%
  select(userID, Rcuisine, present) %>%
  spread(Rcuisine, present, fill=0) %>% na.omit()

output2 <- matrix(ncol=3, nrow=30)
for (i in 1:20) {
  output2[i,1] <- i
  km <- kmeans(horizontal2[2:24], i, nstart=20)
  output2[i,2] <- mean(km$withinss)
  output2[i,3] <- mean(km$betweenss)
}
colnames(output2) <- c('cluster', 'within', 'between')
output2 <- data.frame(output2)

ggplot(output2, aes(x=cluster, y=within)) + geom_point() + theme_classic()
ggplot(output2, aes(x=cluster, y=between)) + geom_point() + theme_classic()

km <- kmeans(horizontal2[2:24], 6, nstart=20)
k_cluster <- km$centers


Here are the two graphs for between sum of squares and within sum of squares:

<img src="cluster_graphs.png"> 

Our data showed that six clusters appeared to be ideal based on our two graphs. 

Below is a table of the cluster characterization that resulted. The characterization is a simple collection of types of food that stood out for that particular cluster compared to other clusters. It was derived by looking at the center of the cluster with 5 nearest points (users) closest to the center. While it should not exclude other types of food that the user may visit and/or rate, it presents a strong argument for the type of food that the user may go for.


Cluster B, for instance, appeared to prefer fast, affordable, take-out food. Cluster E appeared to prefer joints with casual drinks and light food as their primary focus.

<h3><center> Cluster Characterizations </center></h3>

| <br> | Preference 1 | Preference 2 | Preference 3 |
|-|-|-|-|
| Clustser A | Bar | Brewery | <br> |
| Cluster B | Chinese | Fast Food | Pizzeria |
| Cluster C | Armenian | Mexican | Regional |
| Cluster D | American | Burger | Game |
| Cluster E | Bar | Cafe | <br> |
| Cluster F | Family | <br> | <br> |


#### Analyzing Various Clusters

We looked more in detail at characteristics such as age, weight, hijos (number of kids), and budget that determined how each of the clusters were formed. In addition, we graphed the density of each of the clusters with respect to that variable. 

##### Age

Figure 1, the bar plot show the mean age for each of the clusters as well as the variance. While Figure 1 fails to show a significant difference in the mean age, Figure 2 shows the different distribution of the age. Cluster F, for example, shows a wide spread of different age, while cluster C shows a strong peak around age 28 through 31.

<img src="Rplot.png"> 
<i><center>Figure 1</center></i>

<img src="Rplot01.png">
<i><center>Figure 2</center></i>

##### Weight (Kilograms)

Figure 3, similar to Figure 1, does not show a significant difference in mean weight. Figure 4 shows more insight on the distribution of different weight within each cluster. Most clusters seem to have a right skew.

<img src="Rplot02.png">
<i><center>Figure 3</center></i>

<img src="Rplot03.png">
<i><center>Figure 4</center></i>

##### Hijos (number of kids) 

Hijos, in Spanish, means children. In this context, hijos implies a dependency status. Whether the individuals are someone’s dependent, has a dependent, or is considered independent. Most clusters behave similarly. Interestingly, Cluster B, which has a strong preference for fast food, has the highest number of independent individuals. 

<img src="Rplot05.png">
<i><center>Figure 5</center></i>

##### Budget

Budget quite literally considers the budget the individual user has. While similarly distributed, it is interesting to note that Cluster B (fast food) has a significantly higher proportion of low-budget individuals and Cluster C and F (family/cultural restaurants) has a slight tendency towards medium-budget.

<img src="Rplot08.png">
<i><center>Figure 6</center></i>

##### Ambience

Ambience refers to user’s preferred group of people to eat with -family, friends, or by themselves. The distribution seems to not vary much depending on the clusters. Note that ambience was also left out of the OLS regression. 

<img src="Rplot06.png">



## Conclusion

Linear regression is a useful tool for observing the relationship between a set of variables. However, it is important to note that linear regression is sensitive to outliers. In addition, we made the assumption that each of our data points is independent; this is contrary to our assumption in clustering, where we group people by their various food tastes. Through regression, we established that people with different backgrounds - such as family, occupation, or preference for alcohol- had different rating behaviors. This goes to show that the absolute rating system using collective rating may not be beneficial to the individual users.

While k-means clustering is useful for predicting how similar groups individuals are, it is difficult for the algorithm to predict the correct number of clusters (k-value). In addition, k-means clusters are sensitive to scale; rescaling the datasets through normalization or standardization will change the results significantly. We should also note that k-means clustering will always show a cluster of some sort: the clusters that we found may not be the best set of clusters. However, through both visualization and overview of the different behaviors of the clusters, we see that the clusters we found may be a significant determinant of user preference. 

In conclusion, we see that both the linear regression and clusters are good tools for using more context in determining user preference and thus the recommendation for the users. In future steps, we aspire to build a full recommendation system that would pick out a short list of restaurants that the user may enjoy. This would allow us to collect live data on the user and further fine-tune the recommendation system. 


## References

We would like to thank Professor Hongning Wang at the University of Virginia for discussing multi-armed bandit algorithm with us. 